# Movie Recommendation using Knowledge Graphs

# Installation Of Modules

In [1]:
pip install --upgrade py2neo

Note: you may need to restart the kernel to use updated packages.


In [2]:
 pip install --upgrade --pre py2neo

Note: you may need to restart the kernel to use updated packages.


In [3]:
from py2neo import Graph,Node,Relationship

In [4]:
graph = Graph("bolt://localhost:7687", auth=("neo4j", "movie"))

In [5]:
pip install neo4jupyter

Note: you may need to restart the kernel to use updated packages.


In [6]:
!pip install neo4j
from neo4j import GraphDatabase
import neo4jupyter
driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "movie"))


In [7]:
neo4jupyter.init_notebook_mode()

<IPython.core.display.Javascript object>

In [8]:
query="""LOAD CSV WITH HEADERS FROM 'file:///movies.csv' AS line
 MERGE (m:Movie{ id:line.movieId, title:line.title}) 
 FOREACH (gName in split(line.genres, '|') | 
    MERGE (g:Genre {name:gName}) 
    MERGE (m)-[:IS_GENRE]->(g)
)"""
graph.run(query).data()

[]

In [9]:
query="""Match (n:Movie) return n.id as MovieId , n.title as title """
mdf=graph.run(query).to_data_frame()
mdf.head()

,MovieId,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)


In [10]:
query="""MATCH p=()-[r:IS_GENRE]->() RETURN p LIMIT 25"""
mdf=graph.run(query).data()
mdf[0:5]

[{'p': Path(Node('Movie', id='1', title='Toy Story (1995)', tmdbId='862'), IS_GENRE(Node('Movie', id='1', title='Toy Story (1995)', tmdbId='862'), Node('Genre', name='Adventure')))},
 {'p': Path(Node('Movie', id='1', title='Toy Story (1995)', tmdbId='862'), IS_GENRE(Node('Movie', id='1', title='Toy Story (1995)', tmdbId='862'), Node('Genre', name='Animation')))},
 {'p': Path(Node('Movie', id='1', title='Toy Story (1995)', tmdbId='862'), IS_GENRE(Node('Movie', id='1', title='Toy Story (1995)', tmdbId='862'), Node('Genre', name='Children')))},
 {'p': Path(Node('Movie', id='1', title='Toy Story (1995)', tmdbId='862'), IS_GENRE(Node('Movie', id='1', title='Toy Story (1995)', tmdbId='862'), Node('Genre', name='Comedy')))},
 {'p': Path(Node('Movie', id='1', title='Toy Story (1995)', tmdbId='862'), IS_GENRE(Node('Movie', id='1', title='Toy Story (1995)', tmdbId='862'), Node('Genre', name='Fantasy')))}]

In [11]:
query="""LOAD CSV WITH HEADERS FROM "file:///ratings.csv" AS line
 MATCH (m:Movie {id:line.movieId})
 MERGE (u:User {id:line.userId})
 MERGE (u)-[:RATED { rating: toFloat(line.rating)}]->(m);"""
graph.run(query).data()

[]

In [12]:
query="""Match (n:User) return n.id as UserId """
mdf=graph.run(query).to_data_frame()
mdf.head()

,UserId
0,1
1,2
2,3
3,4
4,5


In [13]:
query="""MATCH p=()-[r:RATED]->() RETURN p LIMIT 25"""
mdf=graph.run(query).data()
mdf[0:5]

[{'p': Path(Node('User', id='1'), RATED(Node('User', id='1'), Node('Movie', id='661', title='James and the Giant Peach (1996)', tmdbId='10539'), rating=5.0))},
 {'p': Path(Node('User', id='1'), RATED(Node('User', id='1'), Node('Movie', id='3034', title='Robin Hood (1973)', tmdbId='11886'), rating=5.0))},
 {'p': Path(Node('User', id='1'), RATED(Node('User', id='1'), Node('Movie', id='1473', title='Best Men (1997)', tmdbId='19952'), rating=4.0))},
 {'p': Path(Node('User', id='1'), RATED(Node('User', id='1'), Node('Movie', id='3273', title='Scream 3 (2000)', tmdbId='4234'), rating=5.0))},
 {'p': Path(Node('User', id='1'), RATED(Node('User', id='1'), Node('Movie', id='2387', title='Very Bad Things (1998)', tmdbId='10029'), rating=5.0))}]

In [14]:
query="""LOAD CSV WITH HEADERS FROM "file:///tags.csv" AS line
 MATCH (m:Movie {id:line.movieId})
 MATCH (u:User {id:line.userId})
 CREATE (u)-[:TAGGED { tag: line.tag}]->(m);"""
graph.run(query).data()

[]

In [15]:
query="""MATCH p=()-[r:ACTED_IN]->() RETURN p LIMIT 25 """
mdf=graph.run(query).to_data_frame()
mdf.head()

,p
0,({'role': 'Richard Weidner'})
1,({'role': 'Paulie Romano'})
2,({'role': 'Joey Randone'})
3,"({'role': '""Pistol"" Pete Deeks'})"
4,({'role': 'Wirey Pink'})


In [16]:
query="""MATCH p=()-[r:TAGGED]->() RETURN p LIMIT 25"""
mdf=graph.run(query).data()
mdf[0:5]

[{'p': Path(Node('User', id='2'), TAGGED(Node('User', id='2'), Node('Movie', id='89774', title='Warrior (2011)', tmdbId='59440'), tag='Boxing story'))},
 {'p': Path(Node('User', id='2'), TAGGED(Node('User', id='2'), Node('Movie', id='89774', title='Warrior (2011)', tmdbId='59440'), tag='Boxing story'))},
 {'p': Path(Node('User', id='2'), TAGGED(Node('User', id='2'), Node('Movie', id='106782', title='Wolf of Wall Street, The (2013)', tmdbId='106646'), tag='Leonardo DiCaprio'))},
 {'p': Path(Node('User', id='2'), TAGGED(Node('User', id='2'), Node('Movie', id='60756', title='Step Brothers (2008)', tmdbId='12133'), tag='funny'))},
 {'p': Path(Node('User', id='2'), TAGGED(Node('User', id='2'), Node('Movie', id='89774', title='Warrior (2011)', tmdbId='59440'), tag='Tom Hardy'))}]

In [17]:
query="""LOAD CSV WITH HEADERS FROM "file:///links.csv" AS line
MATCH (m:Movie {id:line.movieId})
SET m.tmdbId=line.tmdbId;"""
graph.run(query).data()

[]

In [18]:
query="""LOAD CSV WITH HEADERS FROM "file:///directors.csv" AS line
MATCH (m:Movie{ tmdbId:line.movieId})
MERGE (p:Person{name:line.person_name})
MERGE (p)-[:DIRECTED]->(m);"""
graph.run(query).data()

[]

In [19]:
query="""LOAD CSV WITH HEADERS FROM "file:///roles.csv" AS line
MATCH (m:Movie{ tmdbId:line.movieId})
MERGE (p:Person{name:line.person_name})
CREATE (p)-[r:ACTED_IN] ->(m)
SET  r.role= line.role;"""
graph.run(query).data()

[]

In [41]:
print("Knowledge graph")
(neo4jupyter.draw(graph,{'Movie':'title','User':'id','Person':'name','Genre':'name'}))


Knowledge graph


In [21]:
query="""MATCH (m:Movie {title: "Toy Story (1995)"})-[:ACTED_IN|:IS_GENRE|:DIRECTED]-(p)
RETURN m.id as Movieid,m.title as title,m.tmdbId as tmdbId, p.name as Genre_Crew_Actor limit 10"""
graph.run(query).to_data_frame()


,Movieid,title,tmdbId,Genre_Crew_Actor
0,1,Toy Story (1995),862,Adventure
1,1,Toy Story (1995),862,Animation
2,1,Toy Story (1995),862,Children
3,1,Toy Story (1995),862,Comedy
4,1,Toy Story (1995),862,Fantasy
5,1,Toy Story (1995),862,John Lasseter
6,1,Toy Story (1995),862,John Ratzenberger
7,1,Toy Story (1995),862,Don Rickles
8,1,Toy Story (1995),862,Laurie Metcalf
9,1,Toy Story (1995),862,Wallace Shawn


In [22]:
query="""MATCH (m:Movie {title: "Toy Story (1995)"})-[:RATED|:TAGGED]-(u)
RETURN m.id as Movieid,m.title as title,m.tmdbId as tmdbId, u.id as Userid LIMIT 10"""
graph.run(query).to_data_frame()

,Movieid,title,tmdbId,Userid
0,1,Toy Story (1995),862,298
1,1,Toy Story (1995),862,161
2,1,Toy Story (1995),862,276
3,1,Toy Story (1995),862,332
4,1,Toy Story (1995),862,399
5,1,Toy Story (1995),862,559
6,1,Toy Story (1995),862,82
7,1,Toy Story (1995),862,223
8,1,Toy Story (1995),862,185
9,1,Toy Story (1995),862,247


In [23]:
query="""MATCH (user:User{id:'220'})-[r1:RATED]->(m:Movie)<-[r2:RATED]-(other:User)-[r3:RATED]->(m2:Movie)
WHERE r1.rating > 3 AND r2.rating > 3 AND r3.rating > 3 AND NOT (user)-[:RATED]->(m2)
RETURN distinct m2.id as Movieid,m2.title as title,m2.tmdbId as tmdbId, count(*) AS score
ORDER BY score DESC
LIMIT 15"""
graph.run(query).to_data_frame()

,Movieid,title,tmdbId,score
0,593,"Silence of the Lambs, The (1991)",274,7203
1,4993,"Lord of the Rings: The Fellowship of the Ring,...",120,6563
2,2858,American Beauty (1999),14,6227
3,110,Braveheart (1995),197,5894
4,3578,Gladiator (2000),98,5777
5,527,Schindler's List (1993),424,5663
6,1136,Monty Python and the Holy Grail (1975),762,5377
7,4963,Ocean's Eleven (2001),161,5011
8,1214,Alien (1979),348,4951
9,1682,"Truman Show, The (1998)",37165,4867


In [24]:
query="""MATCH (user:User{id:'220'})-[r:RATED]-(m)
WITH user, avg(r.rating) AS average
MATCH (user)-[r1:RATED]->(m:Movie)<-[r2:RATED]-(other:User)-[r3:RATED]->(m2:Movie)
WHERE r1.rating > average AND r2.rating > average AND r3.rating > average AND NOT (user)-[:RATED]->(m2)
RETURN distinct m2.id as Movieid,m2.title as title,m2.tmdbId as tmdbId, count(*) AS score
ORDER BY score DESC
LIMIT 15"""
graph.run(query).to_data_frame()

,Movieid,title,tmdbId,score
0,593,"Silence of the Lambs, The (1991)",274,5322
1,4993,"Lord of the Rings: The Fellowship of the Ring,...",120,4276
2,2858,American Beauty (1999),14,4129
3,527,Schindler's List (1993),424,4086
4,110,Braveheart (1995),197,3982
5,3578,Gladiator (2000),98,3537
6,1214,Alien (1979),348,3502
7,1136,Monty Python and the Holy Grail (1975),762,3408
8,1221,"Godfather: Part II, The (1974)",240,3330
9,457,"Fugitive, The (1993)",5503,3222


In [25]:
query="""MATCH (user:User{id:'318'})-[r:RATED]-(m)
WITH user, avg(r.rating) AS average
MATCH (user)-[t1:TAGGED]->(m:Movie)-[r:RATED]-(user)
MATCH (other:User)-[t2:TAGGED]->(m1:Movie)
WHERE r.rating > average AND t1.tag=t2.tag AND  NOT (user)-[:TAGGED]->(m1) AND  NOT (user)-[:RATED]->(m1)
RETURN m1.id as Movieid,m1.title as title,m1.tmdbId as tmdbId, other.id as OtherUserid"""
graph.run(query).to_data_frame()

,Movieid,title,tmdbId,OtherUserid
0,410,Addams Family Values (1993),2758,62
1,410,Addams Family Values (1993),2758,62
2,410,Addams Family Values (1993),2758,62
3,410,Addams Family Values (1993),2758,62
4,410,Addams Family Values (1993),2758,62
...,...,...,...,...
3259,184471,Tomb Raider (2018),338970,62
3260,184471,Tomb Raider (2018),338970,62
3261,184471,Tomb Raider (2018),338970,62
3262,184471,Tomb Raider (2018),338970,62


In [26]:
query="""MATCH (user:User{id:'318'})-[r:RATED]-(m:Movie)
WITH user, avg(r.rating) AS average
MATCH (user)-[r:RATED]->(m:Movie)-[:ACTED_IN]-(p:Person)
WHERE r.rating > average
RETURN  p.name as actor, COUNT(*) AS score 
ORDER BY score DESC LIMIT 10"""
graph.run(query).to_data_frame()


,actor,score
0,Matt Damon,80
1,Johnny Depp,80
2,George Clooney,72
3,Steve Buscemi,64
4,Bill Hader,64
5,Brad Pitt,64
6,Philip Seymour Hoffman,56
7,Sean Penn,56
8,John C. Reilly,56
9,David Cross,48


In [27]:
query="""MATCH (user:User{id:'318'})-[r:RATED]-(m:Movie)
WITH user, avg(r.rating) AS average
MATCH (user)-[r:RATED]->(m:Movie)-[:DIRECTED]-(p:Person)
WHERE r.rating > average
RETURN  p.name as director, COUNT(*) AS score 
ORDER BY score DESC LIMIT 10"""
graph.run(query).to_data_frame()

,director,score
0,Joel Coen,7
1,Christopher Nolan,5
2,Steven Spielberg,5
3,Quentin Tarantino,4
4,Kevin Smith,4
5,Tim Burton,3
6,Wes Anderson,3
7,Edgar Wright,3
8,Larry Charles,3
9,Paul Thomas Anderson,3


In [28]:
query="""MATCH (user:User{id:'318'})-[r:RATED]-(m:Movie)
WITH user, avg(r.rating) AS average
MATCH (user)-[r:RATED]->(m:Movie)-[:IS_GENRE]-(p:Genre)
WHERE r.rating > average
RETURN  p.name as genre, COUNT(*) AS score 
ORDER BY score DESC LIMIT 10"""
graph.run(query).to_data_frame()

,genre,score
0,Drama,232
1,Comedy,150
2,Thriller,77
3,Action,73
4,Crime,72
5,Adventure,66
6,Documentary,61
7,Sci-Fi,49
8,Romance,49
9,Animation,47


In [29]:
query="""MATCH (user:User{id:'318'})-[r:RATED]-(m:Movie)
WITH user, avg(r.rating) AS average
MATCH (user)-[r:RATED]->(m:Movie)
WHERE r.rating > average
MATCH (m)-[:IS_GENRE]->(g:Genre)<-[:IS_GENRE]-(rm:Movie)
WITH user, m, rm, COUNT(*) AS gs
OPTIONAL MATCH (m)<-[:ACTED_IN]-(a:Person)-[:ACTED_IN]->(rm)
WITH user, m, rm, gs, COUNT(a) AS as
OPTIONAL MATCH (m)<-[:DIRECTED]-(d:Person)-[:DIRECTED]->(rm)
WITH user, m, rm, gs, as, COUNT(d) AS ds
MATCH (rm)
WHERE  NOT (user)-[:RATED]->(rm)
RETURN rm.id as MovieId,rm.title AS title,
(4*gs)+(3*as)+(3*ds) AS weighed_score 
ORDER BY weighed_score DESC LIMIT 10"""
graph.run(query).to_data_frame()

,MovieId,title,weighed_score
0,135133,The Hunger Games: Mockingjay - Part 2 (2015),2507
1,104218,Grown Ups 2 (2013),2503
2,78499,Toy Story 3 (2010),2132
3,149406,Kung Fu Panda 3 (2016),2124
4,71264,Cloudy with a Chance of Meatballs (2009),2124
5,78499,Toy Story 3 (2010),1932
6,103335,Despicable Me 2 (2013),1740
7,3160,Magnolia (1999),1735
8,112138,22 Jump Street (2014),1551
9,4734,Jay and Silent Bob Strike Back (2001),1547


In [30]:
query="""MATCH (user:User{id:'318'})-[r:RATED]-(m:Movie)
WITH user, avg(r.rating) AS average
MATCH (user)-[r:RATED]->(m:Movie)
WHERE r.rating > 4.5
MATCH (m)<-[:ACTED_IN]-(a:Person)-[:ACTED_IN]->(rm)
MATCH (rm)
WHERE  NOT (user)-[:RATED]->(rm)
RETURN rm.id as MovieId,rm.title as title,rm.tmdbId as tmdbId, a.name as Actor_name, user.id as userId ,m.id as Movieid,m.title as Movietitle,m.tmdbId as tmdbid LIMIT 10"""
graph.run(query).to_data_frame()

,MovieId,title,tmdbId,Actor_name,userId,Movieid,Movietitle,tmdbid
0,899,Singin' in the Rain (1952),872,Debbie Reynolds,318,1884,Fear and Loathing in Las Vegas (1998),1878
1,3990,Rugrats in Paris: The Movie (2000),16340,Debbie Reynolds,318,1884,Fear and Loathing in Las Vegas (1998),1878
2,3257,"Bodyguard, The (1992)",619,Debbie Reynolds,318,1884,Fear and Loathing in Las Vegas (1998),1878
3,92264,One for the Money (2012),54054,Debbie Reynolds,318,1884,Fear and Loathing in Las Vegas (1998),1878
4,1614,In & Out (1997),10806,Debbie Reynolds,318,1884,Fear and Loathing in Las Vegas (1998),1878
5,92264,One for the Money (2012),54054,Debbie Reynolds,318,1884,Fear and Loathing in Las Vegas (1998),1878
6,3990,Rugrats in Paris: The Movie (2000),16340,Debbie Reynolds,318,1884,Fear and Loathing in Las Vegas (1998),1878
7,3257,"Bodyguard, The (1992)",619,Debbie Reynolds,318,1884,Fear and Loathing in Las Vegas (1998),1878
8,899,Singin' in the Rain (1952),872,Debbie Reynolds,318,1884,Fear and Loathing in Las Vegas (1998),1878
9,1614,In & Out (1997),10806,Debbie Reynolds,318,1884,Fear and Loathing in Las Vegas (1998),1878


In [31]:
#Jacard Index
query="""MATCH (user:User{id:'220'})-[r:RATED]-(m:Movie)
WITH user, avg(r.rating) AS mean
MATCH (user)-[r:RATED]->(m:Movie)
WHERE r.rating =5
MATCH (m)-[:ACTED_IN|:DIRECTED]-(t)-[:ACTED_IN|:DIRECTED]-(other:Movie)
WHERE  NOT (user)-[:RATED]->(other)
WITH user, m, other, COUNT(distinct t) AS intersection, COLLECT(t.name) AS i
MATCH (m)-[:ACTED_IN|:DIRECTED]-(mt)
WITH user, m,other, intersection,i, COLLECT(distinct mt.name) AS s1
MATCH (other)-[:ACTED_IN|:DIRECTED]-(ot)
WITH user, m,other,intersection,i, s1,COLLECT(distinct ot.name) AS s2
WITH user, m,other,intersection,s1,s2
WITH user, m,other,intersection,s1+[x IN s2 WHERE NOT x IN s1] AS union, s1, s2
RETURN m.title as title, other.title as Othertitle, s1 as s1,  s2 as s2,((1.0*intersection)/SIZE(union)) AS jaccard ORDER BY jaccard DESC"""
jdataframe = graph.run(query).to_data_frame()
jdataframe

,title,Othertitle,s1,s2,jaccard
0,Casablanca (1942),Top Hat (1935),[Gino Corrado],[Gino Corrado],1.000000
1,Pirates of the Caribbean: The Curse of the Bla...,Pirates of the Caribbean: At World's End (2007),"[Gore Verbinski, Angus Barnett, Zoe Saldana, G...","[Gore Verbinski, Johnny Depp, Orlando Bloom, C...",0.590909
2,Shrek 2 (2004),Shrek the Third (2007),"[Andrew Adamson, Christopher Knights, Cody Cam...","[Conrad Vernon, Amy Poehler, Rupert Everett, M...",0.520000
3,Casablanca (1942),Gone with the Wind (1939),[Gino Corrado],"[Gino Corrado, Thomas Mitchell]",0.500000
4,Casablanca (1942),Intolerance: Love's Struggle Throughout the Ag...,[Gino Corrado],"[Gino Corrado, Edward Burns]",0.500000
...,...,...,...,...,...
6965,Pulp Fiction (1994),Movie 43 (2013),"[Quentin Tarantino, Uma Thurman, Christopher W...","[Richard Gere, Jimmy Bennett, Gerard Butler, M...",0.017241
6966,Star Trek (2009),Austin Powers in Goldmember (2002),"[J.J. Abrams, Leonard Nimoy, Jeff Chase, Pavel...","[Jay Roach, Tom Lister Jr., Verne Troyer, Kevi...",0.016949
6967,Star Trek (2009),Zoolander (2001),"[J.J. Abrams, Leonard Nimoy, Jeff Chase, Pavel...","[Ben Stiller, Andrew Wilson, Jon Voight, Nora ...",0.016393
6968,Star Trek (2009),"Dark Knight Rises, The (2012)","[J.J. Abrams, Leonard Nimoy, Jeff Chase, Pavel...","[Christopher Nolan, Michael Caine, Reggie Lee,...",0.015625


In [32]:
#Pearson correlation coefficient
query="""MATCH (user:User {id:"220"})-[r:RATED]->(m:Movie)
WITH user, avg(r.rating) AS user_average
MATCH (user)-[r1:RATED]->(m:Movie)<-[r2:RATED]-(other)
WITH user, user_average, other, COLLECT({r1: r1, r2: r2}) AS ratings WHERE size(ratings) > 10
MATCH (other)-[r:RATED]->(m:Movie)
WITH user, user_average, other, avg(r.rating) AS other_average, ratings
UNWIND ratings AS r
WITH sum( (r.r1.rating- user_average) * (r.r2.rating- other_average) ) AS a,
sqrt( sum( (r.r1.rating - user_average)^2) * sum( (r.r2.rating - other_average) ^2)) AS b,
user, other 
WHERE b <> 0
RETURN user.id as UserId, other.id as OtherUid, a/b as correlation
ORDER BY correlation DESC LIMIT 10"""
graph.run(query).to_data_frame()

,UserId,OtherUid,correlation
0,220,494,0.782532
1,220,32,0.781892
2,220,485,0.763311
3,220,97,0.754797
4,220,79,0.739910
5,220,88,0.732811
6,220,124,0.718593
7,220,235,0.716539
8,220,436,0.704376
9,220,500,0.659741


In [33]:
query="""MATCH (user:User {id:"220"})-[:RATED]->(m:Movie)
MATCH (other:User {id:"494"})-[:RATED]->(m:Movie)
RETURN user.id as Userid, other.id as otherUid, m.id as MovieId,m.title as title,m.tmdbId as tmdbId"""
graph.run(query).to_data_frame()

,Userid,otherUid,MovieId,title,tmdbId
0,220,494,858,"Godfather, The (1972)",238
1,220,494,589,Terminator 2: Judgment Day (1991),280
2,220,494,1200,Aliens (1986),679
3,220,494,1198,Raiders of the Lost Ark (Indiana Jones and the...,85
4,220,494,260,Star Wars: Episode IV - A New Hope (1977),11
5,220,494,1210,Star Wars: Episode VI - Return of the Jedi (1983),1892
6,220,494,1036,Die Hard (1988),562
7,220,494,2571,"Matrix, The (1999)",603
8,220,494,1196,Star Wars: Episode V - The Empire Strikes Back...,1891
9,220,494,1291,Indiana Jones and the Last Crusade (1989),89


In [34]:
query="""MATCH (user:User {id:"220"})-[r:RATED]->(m:Movie)
WITH user, avg(r.rating) AS user_average
MATCH (user)-[r1:RATED]->(m:Movie)<-[r2:RATED]-(other)
WITH user, user_average, other, COLLECT({r1: r1, r2: r2}) AS ratings WHERE size(ratings) > 10
MATCH (other)-[r:RATED]->(m:Movie)
WITH user, user_average, other, avg(r.rating) AS other_average, ratings
UNWIND ratings AS r
WITH sum( (r.r1.rating- user_average) * (r.r2.rating- other_average) ) AS a,
sqrt( sum( (r.r1.rating - user_average)^2) * sum( (r.r2.rating - other_average) ^2)) AS b,user, other 
WHERE b <> 0
WITH user, other, a/b as correlation
ORDER BY correlation DESC LIMIT 10
MATCH (other)-[r:RATED]->(m:Movie) WHERE NOT EXISTS( (user)-[:RATED]->(m) )
WITH m,  SUM( correlation* r.rating) AS score, COLLECT(other) AS other
RETURN m.id as MovieId,m.title as title,m.tmdbId as tmdbId, other, score
ORDER BY score DESC LIMIT 10"""
graph.run(query).to_data_frame()

,MovieId,title,tmdbId,other,score
0,593,"Silence of the Lambs, The (1991)",274,"[{'id': '32'}, {'id': '485'}, {'id': '97'}, {'...",20.070192
1,527,Schindler's List (1993),424,"[{'id': '32'}, {'id': '88'}, {'id': '235'}, {'...",13.163388
2,457,"Fugitive, The (1993)",5503,"[{'id': '485'}, {'id': '79'}, {'id': '235'}, {...",12.400923
3,110,Braveheart (1995),197,"[{'id': '494'}, {'id': '485'}, {'id': '124'}, ...",11.583822
4,440,Dave (1993),11566,"[{'id': '32'}, {'id': '235'}, {'id': '436'}, {...",11.450195
5,592,Batman (1989),268,"[{'id': '32'}, {'id': '485'}, {'id': '235'}, {...",10.462137
6,246,Hoop Dreams (1994),14275,"[{'id': '32'}, {'id': '79'}, {'id': '235'}]",9.693276
7,161,Crimson Tide (1995),8963,"[{'id': '485'}, {'id': '235'}, {'id': '436'}]",8.783676
8,539,Sleepless in Seattle (1993),858,"[{'id': '32'}, {'id': '235'}, {'id': '436'}, {...",8.766594
9,3578,Gladiator (2000),98,"[{'id': '494'}, {'id': '97'}, {'id': '79'}]",8.411665


In [35]:
query="""MATCH (user:User {id:"220"})-[r:RATED]->(m:Movie)
WITH user, avg(r.rating) AS user_average
MATCH (user)-[r1:RATED]->(m:Movie)<-[r2:RATED]-(other)
WITH user, user_average, other, COLLECT({r1: r1, r2: r2}) AS ratings WHERE size(ratings) > 10
MATCH (other)-[r:RATED]->(m:Movie)
WITH user, user_average, other, avg(r.rating) AS other_average, ratings
UNWIND ratings AS r
WITH sum( (r.r1.rating- user_average) * (r.r2.rating- other_average) ) AS a,
sqrt( sum( (r.r1.rating - user_average)^2) * sum( (r.r2.rating - other_average) ^2)) AS b,user, other 
WHERE b <> 0
WITH user, other, a/b as correlation
ORDER BY correlation ASC LIMIT 10
MATCH (other)-[r:RATED]->(m:Movie) WHERE NOT EXISTS( (user)-[:RATED]->(m) )
WITH m,  SUM( correlation* r.rating) AS score, COLLECT(other) AS other
RETURN m.id as MovieId,m.title as title,m.tmdbId as tmdbId, other, score
ORDER BY score ASC LIMIT 10"""
graph.run(query).to_data_frame()

,MovieId,title,tmdbId,other,score
0,4973,"Amelie (Fabuleux destin d'Amélie Poulain, Le) ...",194,"[{'id': '23'}, {'id': '416'}, {'id': '471'}, {...",-7.210099
1,527,Schindler's List (1993),424,"[{'id': '395'}, {'id': '416'}, {'id': '471'}, ...",-6.143148
2,750,Dr. Strangelove or: How I Learned to Stop Worr...,935,"[{'id': '23'}, {'id': '416'}, {'id': '212'}, {...",-6.019002
3,3949,Requiem for a Dream (2000),641,"[{'id': '23'}, {'id': '212'}, {'id': '110'}]",-5.239955
4,778,Trainspotting (1996),627,"[{'id': '23'}, {'id': '492'}, {'id': '212'}, {...",-5.138655
5,1199,Brazil (1985),68,"[{'id': '23'}, {'id': '416'}]",-4.380639
6,1222,Full Metal Jacket (1987),600,"[{'id': '23'}, {'id': '110'}]",-4.274487
7,592,Batman (1989),268,"[{'id': '395'}, {'id': '37'}, {'id': '174'}]",-4.177436
8,2858,American Beauty (1999),14,"[{'id': '416'}, {'id': '212'}, {'id': '110'}]",-3.819508
9,520,Robin Hood: Men in Tights (1993),8005,"[{'id': '395'}, {'id': '174'}]",-3.793289


In [36]:
query="""MATCH (user:User {id:"220"})-[r:RATED]->(m:Movie)
WITH user, avg(r.rating) AS user_average
MATCH (user)-[r1:RATED]->(m:Movie)<-[r2:RATED]-(other)
WITH user, user_average, other, COLLECT({r1: r1, r2: r2}) AS ratings WHERE size(ratings) > 10
MATCH (other)-[r:RATED]->(m:Movie)
WITH user, user_average, other, avg(r.rating) AS other_average, ratings
UNWIND ratings AS r
WITH sum( (r.r1.rating- user_average) * (r.r2.rating- other_average) ) AS a,
sqrt( sum( (r.r1.rating - user_average)^2) * sum( (r.r2.rating - other_average) ^2)) AS b,user, other 
WHERE b <> 0 and a/b > 0
WITH user, other, a/b as correlation
ORDER BY correlation ASC LIMIT 10
MATCH (other)-[r:RATED]->(m:Movie) WHERE NOT EXISTS( (user)-[:RATED]->(m) )
WITH m,  SUM( correlation* r.rating) AS score , COLLECT(other) AS other
RETURN m.id as MovieId,m.title as title,m.tmdbId as tmdbId, other, score 
ORDER BY score ASC LIMIT 10"""
graph.run(query).to_data_frame()

,MovieId,title,tmdbId,other,score
0,4310,Pearl Harbor (2001),676,[{'id': '63'}],0.005576
1,1587,Conan the Barbarian (1982),9387,[{'id': '361'}],0.005630
2,2410,Rocky III (1982),1371,[{'id': '384'}],0.006056
3,435,Coneheads (1993),9612,[{'id': '384'}],0.006056
4,2347,"Pope of Greenwich Village, The (1984)",32081,[{'id': '384'}],0.006056
5,2122,Children of the Corn (1984),10823,[{'id': '384'}],0.006056
6,1388,Jaws 2 (1978),579,[{'id': '384'}],0.006056
7,2701,Wild Wild West (1999),8487,[{'id': '384'}],0.006056
8,2411,Rocky IV (1985),1374,[{'id': '384'}],0.006056
9,4276,Lost in America (1985),27223,[{'id': '384'}],0.006056
